In [ ]:
# Take as input an embedded Dataframe, and the google sheets original sample,
# and generate a new sample matching the embedded data,
# and a df to with occurence counts for each specialty/scientific name

In [3]:
import pandas as pd
import os

In [20]:
def generate_sheets(df, info):

    # Create page count df
    id_count = df.groupby('ID').count().to_dict()['Embedding']
    df = pd.DataFrame([id_count]).transpose().reset_index()
    df.columns = ['ID', 'Count']

    info = info[info['ID'].isin(id_count.keys())]
    info = info.reset_index().drop(columns=['index'])

    pages_df = pd.merge(info, df, how ='inner', on = 'ID')
    pages_df.drop(pages_df[pages_df['Count'] == 0].index)
    pages_df['Scientific Name'] = pages_df['Scientific Name'].apply(lambda x: x.split(', '))
    pages_df['Specialty'] = pages_df['Specialty'].apply(lambda x: x.split(', '))
    
    # count occurrences of Scientific Name
    scientific_name_counts = {}
    for names in pages_df['Scientific Name']:
        for name in names:
            if name not in scientific_name_counts:
                scientific_name_counts[name] = 1
            else:
                scientific_name_counts[name] += 1

    # count occurrences of Specialty
    specialty_counts = {}
    for specialties in pages_df['Specialty']:
        for specialty in specialties:
            if specialty not in specialty_counts:
                specialty_counts[specialty] = 1
            else:
                specialty_counts[specialty] += 1

    # create new dataframes with the counts
    scientific_name_df = pd.DataFrame.from_dict(scientific_name_counts, orient='index', columns=['Count'])
    specialty_df = pd.DataFrame.from_dict(specialty_counts, orient='index', columns=['Count'])

    # add column to indicate specialty or scientific name
    scientific_name_df['Type'] = 'Scientific Name'
    specialty_df['Type'] = 'Specialty'

    # merge the two dataframes
    sp_sn_count = pd.concat([scientific_name_df, specialty_df], axis=0, sort=False)
    sp_sn_count = sp_sn_count.reset_index()
    sp_sn_count.columns = ['Word', 'Count', 'Type']
    sp_sn_count = sp_sn_count.sort_values('Count', ascending=False)
    
    return pages_df, sp_sn_count.reset_index().drop(columns=['index'])

In [21]:
df = pd.read_csv('openai\openai\openai_concat_embed_sample_smaller.csv')
info = pd.read_csv('Google Sheets.csv')

x, y = generate_sheets(df, info)

In [23]:
x

,ID,Website,Manufacturer,Scientific Name,Specialty,Count
0,18,http://aconlabs.com,"Acon Laboratories, Inc.",[Pregnancy Rapid Tests],"[Laboratory, Rapid Tests]",70
1,148,http://www.bioservicetunisie.com,BioService Tunisie SARL,"[Blood Transfusion Sets, Infusion Sets, Suctio...","[General Consumables & Devices, General Practi...",58
2,189,https://capenergy.com/es/,"Capenergy Medical, S.L.","[Eye Contour, Regenerative Therapy Devices, Rf...","[Dermatology And Aesthetic Medicine, Gynecolog...",35
3,232,http://www.clearsurgical.com,Clear Surgical Ltd.,[Surgical Lights],"[Operating Room, Surgical Machines & Devices]",29
4,283,http://www.ddcdolphin.com,DDC Dolphin Ltd,"[Bedpans, Bedpan Washer-Disinfectors, Disinfec...","[Hospital Furniture & Logistics, Hospital Infr...",61
...,...,...,...,...,...,...
256,374923,http://Sterilumen.com,Sterilumen,[Air Purifiers],"[Hospital Infrastructure, Infrastructure Gener...",18
257,384666,https://www.imaecmedntek.com/,Imaec Medntek,"[Bandages, Dialysis, Dialyzers, Disinfectants,...","[Dialysis, Emergency Medicine, Medical Clothin...",100
258,384707,https://www.vitago.com.tr/,VITAGO,"[Calcium And Vitamin D, Carnitine, Elderberry,...",[Dietary and Herbal Supplements],36
259,394929,https://www.qrskin.com/home.html,QRSKIN GmbH,[Wound Dressings],[Woundcare & Dressing],37
